In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# 超参数设置
vocab_size = 10000  # 词汇量大小
maxlen = 100 # 序列长度
embedding_dim = 32  # 嵌入维度
state_dim = 64 # 状态向量维度

batch_size = 32
epochs = 4 # Early Stopping避免过拟合

In [3]:
# 构建模型
model = Sequential([
    Input(shape=(maxlen,)), 
    Embedding(input_dim=vocab_size,      # 词汇表大小
             output_dim=embedding_dim  # 嵌入维度
             ),
    SimpleRNN(state_dim, return_sequences=False), # 只输出最后一个状态矩阵
    Dense(1, activation='sigmoid')     # 二分类输出
])

model.summary(0)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 326,273 (1.24 MB)

 Trainable params: 326,273 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

RNN模型的参数数量为：~~32x(32+32)+32=2080，其中最后一个32是bias，偏移量b_h是一个常数向量，它在计算隐藏状态h时被加到加权和上。它的作用是调整神经元的激活阈值。~~
64*(64+32)+64

In [4]:
# 加载数据集
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# 数据预处理：填充/截断序列
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [5]:
# 编译模型
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# 训练模型
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5074 - loss: 0.6959 - val_accuracy: 0.5382 - val_loss: 0.6905
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6236 - loss: 0.6526 - val_accuracy: 0.6348 - val_loss: 0.6479
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8211 - loss: 0.3965 - val_accuracy: 0.5428 - val_loss: 0.8370
Epoch 4/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9088 - loss: 0.2114 - val_accuracy: 0.6464 - val_loss: 0.9522


In [7]:
# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss:.4f}')
print(f'\nTest accuracy: {test_acc:.4f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6285 - loss: 1.0179
Test loss: 1.0242

Test accuracy: 0.6280


使用视频中的参数，我实现的效果不是很好。